In [5]:
from dotenv import load_dotenv
load_dotenv()
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [6]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tools
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from openai import OpenAI
import os, requests

In [8]:
llm1= ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                             google_api_key=os.getenv('GEMINI_API_KEY'))
llm = ChatOpenAI(model= "gpt-3.5-turbo", 
                 api_key=os.getenv('GPT_API_KEY'))

I0000 00:00:1722946757.870825 1364763 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722946757.871593 1364763 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


# ImageGenerate

In [ ]:
def generateimage(chapter_content):
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    
    response = client.images.generate(
        model="dall-e-3",
        prompt=f"이미지의 내용: {chapter_content}. Style: Illustration. Create an illustration incorporating a vivid palette with an emphasis on shades of azure and emerald, augmented by splashes of gold for contrast and visual interest. The style should evoke the intricate detail and whimsy of early 20th-century storybook illustrations, blending realism with fantastical elements to create a sense of wonder and enchantment. The composition should be rich in texture, with a soft, luminous lighting that enhances the magical atmosphere. Attention to the interplay of light and shadow will add depth and dimensionality, inviting the viewer to delve into the scene. DON'T include ANY text in this image. DON'T include colour palettes in this image.",
        size="1024x1024",
        quality="standard",
        n=1,
    )

    

    image_url = response.data[0].url
    filename = f"image_{chapter_content[:10].replace(' ', '_')}.png"
    filepath = os.path.join(os.path.dirname(os.getcwd()), 'img', filename)

    image_response = requests.get(image_url)
    if image_response.status_code == 200:
        with open(filepath, 'wb') as file:
            file.write(image_response.content)
    else:
        print("Failed to download the image.")
        return ""

    return filepath


In [5]:
class ImageGeneratorAgent(Agent):
    print("여기가 오류입니다. 1")
    def execute(self, chapter_content):
        image_path = generateimage(chapter_content)
        print(f"이미지 저장 위치 : {image_path}")
        return image_path

여기가 오류입니다. 1


# Agent

In [6]:
story_agent = Agent(
    role='이야기 생성자',
    goal='조선시대를 배경으로 흥미로운 상황을 생성합니다.',
    memory=True,
    backstory='당신은 조선시대의 유명한 이야기꾼입니다. 이야기를 흥미롭게 만들어냅니다.',
    allow_delegation=False,
    llm=llm1
)

# 선택지 에이전트: 상황에 맞는 4가지 선택지를 제공
choice_agent = Agent(
    role='선택지 제공자',
    goal='주어진 상황에 맞는 4가지 선택지를 생성합니다.',
    memory=True,
    backstory='당신은 상황에 맞게 선택지를 제시하는 역할을 맡고 있습니다.',
    allow_delegation=False,
    llm=llm1
)

# 이미지 생성 에이전트 정의
image_generator_agent = Agent(
    role='이미지 생성기',
    goal='이야기에 맞는 이미지를 생성합니다.',
    memory=True,
    backstory='당신은 이야기의 이미지를 생성하는 전문 AI입니다.',
    allow_delegation=False,
    llm=llm1
)


# summerize_agent = Agent(
#     role='내용 요약자',
#     goal='주어진 내용을 읽고 주요 내용을 요약하세요',
#     memory=True,
#     backstory='당신은 내용을 읽고 요약하는 역할을 맡고 있습니다.',
#     allow_delegation=False,
#     llm=llm1
# )

# Tasks

In [7]:
# 이야기 생성 작업
story_creation_task = Task(
    description='조선시대를 배경으로 흥미로운 상황을 생성하세요.'
                '만약 처음이라면 흥미로운 상황을 임의로 생성하세요.'
                '처음이 아니라면 이전의 내용과 이어지도록 상황을 생성하세요'
                '{previous_story}에 이어지는 새로운 상황을 생성하세요',
    expected_output='200글자 내외의 흥미로운 상황이 생성됩니다.',
    agent=story_agent
)

# 선택지 생성 작업
choice_creation_task = Task(
    description='이야기 상황에 맞는 4가지 선택지를 생성하세요. 각 선택지는 이야기와 관련이 있어야 합니다.'
                "{previous_story}와 이어지며, 마지막으로 제시된 상황에 맞는 선택지를 생성하세요"
                "선택지는 1.~ 2.~ 3.~ 4.~ 순으로 제공되고 선택지마다 줄바꿈을 해서 출력하세요",
    expected_output='상황에 맞는 4가지 선택지가 생성됩니다.',
    agent=choice_agent
)

#이미지 생성 작업
task_image_generation = Task(
    description='이야기 내용에 따라 이미지를 생성합니다.',
    agent=image_generator_agent,
    expected_output='생성된 이미지를 반환합니다.'
)

# story_summerization_task = Task(
#     description= '{previous_story}를 읽고 내용을 요약하세요'
#                  '글은 이야기, 선택지, 사용자 선택으로 나누어져있습니다. 이야기에 맞춰 사용자가 선택한 선택지를 참고하여 내용을 요약하세요'
#                  '중요 내용을 판단하여 요약합니다.',
#     expected_output='주어진 글을 읽고 내용을 요약하세요',
#     agent=summerize_agent
# )

# Crew

In [8]:
story_crew = Crew(
    agents=[story_agent, 
            choice_agent, 
            #summerize_agent
            ],
    tasks=[story_creation_task, 
           choice_creation_task, 
           #story_summerization_task
           ],
    process=Process.sequential,
    verbose=True
)

In [11]:


def execute_story(crew, 
                  logfile='log.md', 
                  storyfile = 'story.md',
                  #summary = 'summary.md',
                  #generated_story = 'generated_story.md'
                  ):

    for i in range(9):
        # 요약을 읽기 / 이야기 생성 / 문장 저장  
        story_result = story_creation(crew, storyfile,logfile,i)

        # 이미지 생성 부분 result가 파일 위치
        #result = generateimage(story_result)
        
        # 선택지 생성 / 선택지 내용 저장
        choices_creation = create_options(crew, storyfile, logfile)

        # 사용자 선택 / 선택한 내용 저장
        user_selection(choices_creation, storyfile, logfile, i)
        
        # 내용 가져오기 / 내용 요약 / 요약 파일 누적 저장
        # summerize_contents(crew, storyfile, logfile, summary)
        

        
#이야기 생성 함수
def story_creation(crew, storyfile,logfile,i):
        #요약 파일 읽고 이야기 생성
        with open(storyfile, 'r', encoding='utf-8') as file: 
            previous_story = file.read() 

        crew.tasks = [story_creation_task]
        story_result = crew.kickoff(inputs={'previous_story': previous_story})
        story_text = f"## 상황 {i+1}:\n{story_result}\n\n"

        # 생성된 이야기 log.md 및 story.md, generated_story.md 에저장
        mode = 'w' if i == 0 else 'a' # 처음 생성된 문장의 경우 덮어쓰기 진행
        with open(logfile, mode, encoding='utf-8') as file:
            file.write(story_text)
        with open(storyfile, mode, encoding='utf-8') as file:
            file.write(story_text)
        
        return story_result

#선택지 생성 함수
def create_options(crew, storyfile, logfile):
        #요약 파일 및 방금 생성된 이야기
        with open(storyfile, 'r', encoding="utf") as file:
            previous_story = file.read()


        crew.tasks = [choice_creation_task]
        choices_creation = crew.kickoff(inputs={'previous_story': previous_story})
        story_text = f"{choices_creation}\n\n\n"

        #로그에 저장
        with open(logfile, 'a', encoding='utf-8') as file:
            file.write(story_text)
        
        return choices_creation

#사용자 선택 함수
def user_selection(choices_creation, storyfile, logfile, i):
        
        choice = int(input("1부터 4까지의 숫자 중 하나를 선택하세요: "))
        story_text = f"## 사용자 선택 : {choice}\n\n\n"
        
        ## 숫자 추출
        choices_lines = choices_creation.split("\n")

        line = ""
        for i in choices_lines:
            if i.startswith(f"{choice}."):
                line = f"{i}\n\n\n"
            
        #로그에 사용자가 선택한 숫자 추가
        with open(logfile, 'a', encoding='utf-8') as file:
            file.write(story_text)
        
        #story.md 에 선택한 내용 추가
        with open(storyfile, 'a', encoding='utf-8') as file:
            file.write(line)



# #내용 요약 
# def summerize_contents(crew, storyfile, logfile, summary):
#         with open(storyfile, 'r', encoding="utf") as file:
#             previous_story = file.read()

#         crew.tasks = [story_summerization_task]
#         summerize_story = crew.kickoff(inputs={'previous_story': previous_story})
        
#         # 요약 결과를 파일에 저장
#         with open(summary, 'w', encoding='utf-8') as file:
#              file.write(summerize_story) 

# 크루 실행 및 결과 출력
execute_story(story_crew)

 [DEBUG]: == Working Agent: 이야기 생성자
 [INFO]: == Starting Task: 조선시대를 배경으로 흥미로운 상황을 생성하세요.만약 처음이라면 흥미로운 상황을 임의로 생성하세요.처음이 아니라면 이전의 내용과 이어지도록 상황을 생성하세요## 상황 1:
궁녀는 떨리는 손으로 쪽지를 왕에게 전달했다. 왕은 쪽지를 펼쳐 읽다가 갑자기 얼굴이 새파랗게 질렸다.  "이게 대체 무슨 짓이냐!" 왕의 목소리가 쩌렁쩌렁 울렸다. 쪽지에는 왕의 측근이 반란을 계획하고 있다는 내용이 적혀 있었기 때문이다. 왕은 즉시 친위대를 소집하여 반란을 진압하기 위한 작전을 세우기 시작했다.

2. 측근에게 반란을 중단할 것을 설득하고, 진심으로 회개하면 용서해줄 것을 약속한다. 측근과의 관계 회복을 시도하고, 피비린내 나는 처벌을 피하고자 하는 선택이다.


## 상황 2:
왕의 측근은 뜻밖의 제안에 혼란스러운 표정을 지었다. 그는 왕의 분노에 떨며 몸을 움츠렸지만,  왕의 눈빛에서 진심 어린 설득을 읽어냈다.  "하지만… 이미 많은 사람들이…."  그는 말을 잇지 못했다.  왕은 측근의 어깨를 가볍게 두드리며  "두려워 말게나.  나는 그대를 믿고 있네.  다만, 이제는 나와 함께 나라를 위해 힘을 합쳐야 하네."  라고 말했다.  측근은 왕의 말에 마음이 흔들렸다.  그는 과연 왕의 제안을 받아들여 반란을 중단하고,  왕과 함께 나라를 위해 힘을 합칠 것인가?  아니면,  이미 깊이 빠져든 반란의 길을 끝까지 걸어갈 것인가?  궁궐 안에는 긴장감이 감돌았고,  모두 측근의 결정을 지켜보며 가슴을 조였다.

에 이어지는 새로운 상황을 생성하세요
 [DEBUG]: == [이야기 생성자] Task output: 측근은




I0000 00:00:1722946021.195322 1352918 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


FileNotFoundError: [Errno 2] No such file or directory: '/Users/rohhail/Workspace/crew_ai/crewAI/img/image_측근은.png'